In [11]:
import os
import re
import json
import shutil

from tqdm import tqdm

# Load reports from MIMIC-CXR

In [3]:
docs = []
num_patient = 0
num_study = 0
for dir_path, dir_names, file_names in tqdm(os.walk("/Users/liao/Desktop/mimic-cxr/files")):
    num_patient += 1
    pid = dir_path.split("/")[-1]
    if file_names:
        for file_name in file_names:
            if ".DS_Store" != file_name:
                num_study += 1
                sid = file_name.removesuffix(".txt")
                file_path = os.path.join(dir_path, file_name)
                with open(file_path, "r") as f:
                    doc_lines = f.readlines()
                    docs.append({"pid": pid, "sid": sid, "doc_lines": doc_lines})
print(f"Totally processed: {num_patient} patients | {num_study} studies")

65399it [00:52, 1243.99it/s]

Totally processed: 65399 patients | 227835 studies


# Write to json file

In [12]:
output_dir = "/Users/liao/myProjects/VSCode_workspace/cxr_graph/graph_annotation_process/outputs/mimic"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

os.makedirs(output_dir)

In [13]:
for doc in tqdm(docs):
    output_dict = {
        "doc_key": f"{doc['pid'][:3]}_{doc['pid']}_{doc['sid']}.txt",
        "sentences": [],
    }
    sent_idx = 0
    sent = []
    doc_str = " ".join(doc["doc_lines"])
    toks = re.sub('(?<! )(?=[/,-,:,.,!?()])|(?<=[/,-,:,.,!?()])(?! )', r' ', doc_str).split()
    for tok_id, tok in enumerate(toks):
        sent.append(tok)
        if tok == "." or tok_id == len(toks) - 1:
            output_dict["sentences"].append(sent)
            sent_idx += 1
            sent = []
    with open(os.path.join(output_dir, "inference.json"), "a", encoding="utf-8") as f:
        f.write(json.dumps(output_dict))
        f.write("\n")

print(f"Totally processed: {len(docs)} reports")

100%|██████████| 227835/227835 [00:50<00:00, 4471.73it/s]

Totally processed: 227835 reports
